<h1>Partie 1: Importation des différents package<h1>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

<h1>Partie 2.1: Traitement des données<h1>

<h2>Importation des données sources et anonymes<h2>

In [2]:
N = 100_000_000
separator = '\t'
base_folder = 'data/'
group_folder = 'Moi/'
file = "1"

dsource_file = base_folder + 'donnees_sources'
dtarget_file = base_folder + group_folder + 'donnees_anonymes' + file

attack_file = base_folder + group_folder + 'fichier_attaque' + file + '.json'

In [5]:
dsource = pd.read_csv(dsource_file, nrows=N, sep=separator)
dsource.head(10)

,1,2015-03-04 00:35:16,4.87014666666667,45.77214
0,1,2015-03-04 00:35:48,4.870218,45.772095
1,1,2015-03-04 00:35:49,4.870210,45.772072
2,1,2015-03-04 00:35:50,4.870210,45.772072
3,1,2015-03-04 00:35:52,4.870210,45.772072
4,1,2015-03-04 00:35:54,4.870210,45.772072
5,1,2015-03-04 00:35:55,4.870210,45.772072
6,1,2015-03-04 00:35:56,4.870210,45.772072
7,1,2015-03-04 00:35:58,4.870210,45.772072
8,1,2015-03-04 00:35:59,4.870210,45.772072
9,1,2015-03-04 00:36:01,4.870210,45.772072


In [4]:
dtarget = pd.read_csv(dtarget_file, nrows=N, sep=separator)
dtarget.head(10)

,0,69,2015-03-10 06:53:21,4.59,45.82,DEL
0,1,66,2015-03-20 20:49:17,4.84,45.73,NaN
1,2,68,2015-03-08 02:00:09,2.16,41.41,DEL
2,3,24,2015-05-11 03:28:42,4.86,45.76,NaN
3,4,29,2015-03-28 12:35:21,4.86,45.76,NaN
4,5,87,2015-03-09 20:47:15,4.87,45.78,DEL
5,6,36,2015-03-09 17:35:32,4.87,45.78,DEL
6,7,9,2015-03-05 19:58:04,4.87,45.76,DEL
7,8,87,2015-04-13 19:18:54,5.15,45.63,NaN
8,9,67,2015-04-27 11:07:23,4.88,45.79,NaN
9,10,89,2015-03-07 23:18:28,4.84,45.77,NaN


<h2>Mise en forme des données<h2>

In [8]:
dsource.columns = ['id', 'date', 'lat', 'long']

dsource['id'] = dsource['id'].astype('int64')

dsource['lat'] = pd.to_numeric(dsource['lat']).clip(lower=-90, upper=90)
dsource['long'] = pd.to_numeric(dsource['long']).clip(lower=-180, upper=180)

dsource['date'] = pd.to_datetime(dsource['date'])
dsource['timestamp'] = (dsource['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

dsource['week_year'] = dsource['date'].dt.strftime('%Y-%U')

dsource.head(10)

,id,date,lat,long,week_year,year,week
0,1,2015-03-04 00:35:48,4.870218,45.772095,2015-09,2015,09
1,1,2015-03-04 00:35:49,4.870210,45.772072,2015-09,2015,09
2,1,2015-03-04 00:35:50,4.870210,45.772072,2015-09,2015,09
3,1,2015-03-04 00:35:52,4.870210,45.772072,2015-09,2015,09
4,1,2015-03-04 00:35:54,4.870210,45.772072,2015-09,2015,09
5,1,2015-03-04 00:35:55,4.870210,45.772072,2015-09,2015,09
6,1,2015-03-04 00:35:56,4.870210,45.772072,2015-09,2015,09
7,1,2015-03-04 00:35:58,4.870210,45.772072,2015-09,2015,09
8,1,2015-03-04 00:35:59,4.870210,45.772072,2015-09,2015,09
9,1,2015-03-04 00:36:01,4.870210,45.772072,2015-09,2015,09


In [9]:
for col in ['id', 'date', 'lat', 'long', 'week_year']:
    print(col, dsource[col].dtype)

id int64
date datetime64[ns]
lat float64
long float64
week_year object


In [11]:
dtarget.columns = ['id', 'date', 'lat', 'long']
dtarget = dtarget[~(dtarget == 'DEL').any(axis=1)]

dtarget['id'] = dtarget['id'].astype('int64')

dtarget['lat'] = pd.to_numeric(dtarget['lat'], errors='coerce').clip(lower=-90, upper=90)
dtarget['long'] = pd.to_numeric(dtarget['long'], errors='coerce').clip(lower=-180, upper=180)

dtarget['date'] = pd.to_datetime(dtarget['date'])
dtarget['timestamp'] = (dtarget['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

dtarget['week_year'] = dtarget['date'].dt.strftime('%Y-%U')

dtarget.head(10)

,id,date,lat,long,timestamp,week_year
0,66,2015-03-20 20:49:17,4.84,45.73,1426884557,2015-11
2,24,2015-05-11 03:28:42,4.86,45.76,1431314922,2015-19
3,29,2015-03-28 12:35:21,4.86,45.76,1427546121,2015-12
7,87,2015-04-13 19:18:54,5.15,45.63,1428952734,2015-15
8,67,2015-04-27 11:07:23,4.88,45.79,1430132843,2015-17
9,89,2015-03-07 23:18:28,4.84,45.77,1425770308,2015-09
13,42,2015-05-01 15:20:28,4.76,46.00,1430493628,2015-17
16,32,2015-03-27 19:23:37,4.93,45.78,1427484217,2015-12
19,51,2015-03-18 06:01:41,4.87,45.79,1426658501,2015-11
20,55,2015-03-17 16:34:05,4.93,45.78,1426610045,2015-11


In [12]:
for col in ['id', 'date', 'lat', 'long', 'week_year']:
    print(col, dtarget[col].dtype)

id int64
date datetime64[ns]
lat float64
long float64
week_year object


<h2>Enregistrement des données<h2>

In [19]:
dsource.to_pickle(dsource_file + '.pkl')

In [14]:
dtarget.to_pickle(dtarget_file + '.pkl')

<h1>Partie 2.2: Chargement des données (optionnel si les données sont déjà chargées)<h1>

In [15]:
dsource = pd.read_pickle(dsource_file + '.pkl')

In [12]:
dtarget = pd.read_pickle(dtarget_file + '.pkl')

<h1>Partie 3: Algorithme PCA et KNN<h1>

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

features = ['timestamp', 'lat', 'long']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['timestamp', 'lat', 'long'])
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=3))
])

In [20]:
pipeline.fit(dsource[features])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['timestamp', 'lat',
                                                   'long'])])),
                ('pca', PCA(n_components=3))])

In [21]:
explained_variance = pipeline.named_steps['pca'].explained_variance_ratio_
print("Explained Variance Ratio:", explained_variance)

Explained Variance Ratio: [0.44418831 0.33734834 0.21846335]


In [22]:
source_embedding = pipeline.transform(dsource[features])

In [23]:
target_embedding = pipeline.transform(dtarget[features])

In [25]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=1)
nn.fit(source_embedding)
_, index_list = nn.kneighbors(target_embedding)

<h2>Enregistrement de la liste d'index<h2>

In [26]:
np.save(base_folder + group_folder + 'index_list.npy', index_list)

<h2>Chargement de la liste d'index (optionnel si la liste est déjà chargée)<h2>

In [ ]:
index_list = np.load(base_folder + group_folder + 'index_list.npy')

<h1>Partie 4: Corrélation entre les indices des données sources et des données anonymes<h1>

Récupération des indices sources en fonction des résultats du PCA

In [27]:
dsource_ids = dsource.loc[index_list.squeeze(), 'id']
dsource_ids.head(10)

25817607    77
29084580    72
7721575     42
3735304      6
11160188    26
2113912      9
31718125    87
10286271    13
26335882    69
4180571      7
Name: id, dtype: int64

Ajout de la colonne de la correspondance des indices

In [28]:
dtarget['source_id'] = np.array(dsource_ids)

df = dtarget[['id', 'week_year', 'source_id']]
df.head(10)

,id,week_year,source_id
0,66,2015-11,77
2,24,2015-19,72
3,29,2015-12,42
7,87,2015-15,6
8,67,2015-17,26
9,89,2015-09,9
13,42,2015-17,87
16,32,2015-12,13
19,51,2015-11,69
20,55,2015-11,7


Calcul du nombre d'occurence du tuple (id, week_year, source_id)

In [29]:
occurrences = df.groupby(['id', 'week_year', 'source_id']).size().reset_index(name='occurrences')
df = df.merge(occurrences, on=['source_id', 'week_year', 'id'], how='left')
df.head(10)

,id,week_year,source_id,occurrences
0,66,2015-11,77,14045
1,24,2015-19,72,23175
2,29,2015-12,42,38209
3,87,2015-15,6,128122
4,67,2015-17,26,4430
5,89,2015-09,9,23974
6,42,2015-17,87,40290
7,32,2015-12,13,16428
8,51,2015-11,69,306
9,55,2015-11,7,120943


Calcul du nombre total d'apparition de chaque source_id

In [30]:
total_count = df.groupby('source_id').size().reset_index(name='total_count')
df = df.merge(total_count, on='source_id', how='left')
df.head(10)

,id,week_year,source_id,occurrences,total_count
0,66,2015-11,77,14045,57287
1,24,2015-19,72,23175,977898
2,29,2015-12,42,38209,817150
3,87,2015-15,6,128122,854757
4,67,2015-17,26,4430,195313
5,89,2015-09,9,23974,267784
6,42,2015-17,87,40290,404851
7,32,2015-12,13,16428,408071
8,51,2015-11,69,306,1138162
9,55,2015-11,7,120943,580731


Calcul du pourcentage d'apparition d'un id source pour chaque semaine

In [31]:
df["percentage"] = df["occurrences"] / df["total_count"]
df = df.drop(['occurrences', 'total_count'], axis=1)
df.head(10)

,id,week_year,source_id,percentage
0,66,2015-11,77,0.245169
1,24,2015-19,72,0.023699
2,29,2015-12,42,0.046759
3,87,2015-15,6,0.149893
4,67,2015-17,26,0.022682
5,89,2015-09,9,0.089527
6,42,2015-17,87,0.099518
7,32,2015-12,13,0.040258
8,51,2015-11,69,0.000269
9,55,2015-11,7,0.208260


Suppression des duplicatas

In [32]:
df = df.drop_duplicates()
df.head(10)

,id,week_year,source_id,percentage
0,66,2015-11,77,0.245169
1,24,2015-19,72,0.023699
2,29,2015-12,42,0.046759
3,87,2015-15,6,0.149893
4,67,2015-17,26,0.022682
5,89,2015-09,9,0.089527
6,42,2015-17,87,0.099518
7,32,2015-12,13,0.040258
8,51,2015-11,69,0.000269
9,55,2015-11,7,0.208260


Supression des lignes où le tuple (id, week_year) est le moins présent

In [33]:
max_indices = df.groupby(['id', 'week_year'])['percentage'].idxmax()
result_df = df.loc[max_indices]
result_df.head(10)

,id,week_year,source_id,percentage
411,1,2015-09,89,0.112566
105,1,2015-10,11,0.300762
297,1,2015-11,69,0.034841
147,1,2015-12,43,0.052431
473,1,2015-13,8,0.126757
286,1,2015-14,68,0.065265
413,1,2015-15,27,0.090753
6236,1,2015-16,17,0.071305
250260,1,2015-17,75,0.000089
10096,1,2015-18,30,0.023922


Supression des lignes où le tuple (source_id, week_year) est le moins présent

In [34]:
max_indices = result_df.groupby(['source_id', 'week_year'])['percentage'].idxmax()
result_df = result_df.loc[max_indices]
result_df.head(10)

,id,week_year,source_id,percentage
41,41,2015-09,1,0.023373
140,16,2015-11,1,0.030667
161,68,2015-12,1,0.048047
326,28,2015-13,1,0.046427
110,2,2015-14,1,0.150469
40,9,2015-15,1,0.130931
249,59,2015-16,1,0.059095
176,18,2015-17,1,0.087363
1150,75,2015-18,1,0.051043
1459,51,2015-19,1,0.019341


<h1>Partie 5: Transformation du dataset sous forme de json<h1>

In [36]:
import json

transformed_data = {}

for index, row in result_df.iterrows():
    id_ = str(row['id'])
    week = row['week_year']
    source_id = row['source_id']

    if source_id not in transformed_data:
        transformed_data[source_id] = {}
    if week not in transformed_data[source_id]:
        transformed_data[source_id][week] = []

    transformed_data[source_id][week].append(id_)

result_json = json.dumps(transformed_data, indent=2)

In [37]:
with open(attack_file, 'w') as file:
    file.write(result_json)